In [1]:
import pandas as pd
import os
import gzip
from tqdm.autonotebook import tqdm
tqdm.pandas()
from collections import defaultdict

C:\Users\felix\Anaconda3\lib\site-packages\tqdm\autonotebook\__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [ ]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
root_path = os.path.join(os.getcwd(), 'data', 'data', 'amazon')

aspects = [
    'Apps_for_Android',
    'Baby',
    'Beauty',
    'CDs_and_Vinyl',
    'Cell_Phones_and_Accessories',
    'Clothing_Shoes_and_Jewelry',
    'Digital_Music',
    'Electronics',
    'Grocery_and_Gourmet_Food',
    'Health_and_Personal_Care',
    'Home_and_Kitchen',
    'Kindle_Store',
    'Movies_and_TV',
    'Office_Products',
    'Pet_Supplies',
    'Sports_and_Outdoors',
    'Tools_and_Home_Improvement',
    'Toys_and_Games',
    'Video_Games',
    'Books'
]

In [ ]:
df = None

for a in tqdm(aspects):
    fn = f'reviews_{a}_5.json.gz'
    path = os.path.join(root_path, fn)
    print('Parse ' + path)
    
    a_df = getDF(path)
    a_df = process_df(a_df, a)
    a_df.to_pickle(os.path.join(root_path, a + '_processed.pkl'))
    if df is None:
        df = a_df
    else:
        df = df.append(a_df)

In [ ]:
df.to_csv(os.path.join(root_path, 'dataset_processed.csv'))
df.to_pickle(os.path.join(root_path, 'dataset_processed.pkl'))

In [ ]:
df.groupby('aspect').count()

In [ ]:
#df.sentiment.describe()

In [26]:
path = os.path.join(os.getcwd(), 'data', 'data', 'amazon', 'raw')
df = pd.read_pickle(os.path.join(path, 'dataset_processed.pkl'))
#df = pd.read_pickle(os.path.join(path, 'dataset_processed.pkl'))
df

,helpful,reviewText,overall,summary,sentiment,aspect
1,"[0, 0]","Oh, how my little grandson loves this app. He'...",5.0,2-year-old loves it,positive,Apps_for_Android
2,"[0, 0]",I found this at a perfect time since my daught...,5.0,Fun game,positive,Apps_for_Android
3,"[3, 4]",My 1 year old goes back to this game over and ...,5.0,We love our Monkeys!,positive,Apps_for_Android
4,"[1, 1]",There are three different versions of the song...,5.0,This is my granddaughters favorite app on my K...,positive,Apps_for_Android
5,"[3, 3]",THis is just so cute and a great app for littl...,5.0,so cute,positive,Apps_for_Android
6,"[3, 4]",I watch my great grandson 4 days a week and it...,5.0,Terrific!,positive,Apps_for_Android
7,"[0, 0]",This app is wild and crazy. Little ones love ...,5.0,Five Little Monkeys,positive,Apps_for_Android
8,"[3, 4]",love love love this app. I was going through d...,5.0,love but to quite,positive,Apps_for_Android
9,"[0, 0]","Very cute, with alot of items to move about. ...",5.0,Cute,positive,Apps_for_Android
10,"[1, 2]",WELL THE CHILDREN LOVE IT AFTER AWHILE YOU GET...,4.0,MONKEYS,positive,Apps_for_Android


# Data Exploration

In [27]:
df.groupby('overall').count()

,helpful,reviewText,summary,sentiment,aspect
overall,,,,,
1.0,166232,166232,166232,166232,166232
2.0,171147,171147,171147,171147,171147
3.0,349677,349677,349677,349677,349677
4.0,133262,133262,133262,133262,133262
5.0,372940,372940,372940,372940,372940


In [28]:
df.groupby('sentiment').count()

,helpful,reviewText,overall,summary,aspect
sentiment,,,,,
negative,337379,337379,337379,337379,337379
neutral,349677,349677,349677,349677,349677
positive,506202,506202,506202,506202,506202


## Helpfulnes and word counts

In [29]:
def get_helpfullness(row):
    h = row['helpful']
    if h[1] == 0:
        return 0
    
    return h[0] / h[1]

df['helpfullness'] = df.progress_apply(get_helpfullness, axis=1)
df

A Jupyter Widget

,helpful,reviewText,overall,summary,sentiment,aspect,helpfullness
1,"[0, 0]","Oh, how my little grandson loves this app. He'...",5.0,2-year-old loves it,positive,Apps_for_Android,0.000000
2,"[0, 0]",I found this at a perfect time since my daught...,5.0,Fun game,positive,Apps_for_Android,0.000000
3,"[3, 4]",My 1 year old goes back to this game over and ...,5.0,We love our Monkeys!,positive,Apps_for_Android,0.750000
4,"[1, 1]",There are three different versions of the song...,5.0,This is my granddaughters favorite app on my K...,positive,Apps_for_Android,1.000000
5,"[3, 3]",THis is just so cute and a great app for littl...,5.0,so cute,positive,Apps_for_Android,1.000000
6,"[3, 4]",I watch my great grandson 4 days a week and it...,5.0,Terrific!,positive,Apps_for_Android,0.750000
7,"[0, 0]",This app is wild and crazy. Little ones love ...,5.0,Five Little Monkeys,positive,Apps_for_Android,0.000000
8,"[3, 4]",love love love this app. I was going through d...,5.0,love but to quite,positive,Apps_for_Android,0.750000
9,"[0, 0]","Very cute, with alot of items to move about. ...",5.0,Cute,positive,Apps_for_Android,0.000000
10,"[1, 2]",WELL THE CHILDREN LOVE IT AFTER AWHILE YOU GET...,4.0,MONKEYS,positive,Apps_for_Android,0.500000


In [30]:
df['words'] = df.progress_apply(lambda r: len(r['reviewText'].split(' ')), axis=1)
df

A Jupyter Widget

,helpful,reviewText,overall,summary,sentiment,aspect,helpfullness,words
1,"[0, 0]","Oh, how my little grandson loves this app. He'...",5.0,2-year-old loves it,positive,Apps_for_Android,0.000000,47
2,"[0, 0]",I found this at a perfect time since my daught...,5.0,Fun game,positive,Apps_for_Android,0.000000,53
3,"[3, 4]",My 1 year old goes back to this game over and ...,5.0,We love our Monkeys!,positive,Apps_for_Android,0.750000,43
4,"[1, 1]",There are three different versions of the song...,5.0,This is my granddaughters favorite app on my K...,positive,Apps_for_Android,1.000000,134
5,"[3, 3]",THis is just so cute and a great app for littl...,5.0,so cute,positive,Apps_for_Android,1.000000,31
6,"[3, 4]",I watch my great grandson 4 days a week and it...,5.0,Terrific!,positive,Apps_for_Android,0.750000,60
7,"[0, 0]",This app is wild and crazy. Little ones love ...,5.0,Five Little Monkeys,positive,Apps_for_Android,0.000000,47
8,"[3, 4]",love love love this app. I was going through d...,5.0,love but to quite,positive,Apps_for_Android,0.750000,39
9,"[0, 0]","Very cute, with alot of items to move about. ...",5.0,Cute,positive,Apps_for_Android,0.000000,26
10,"[1, 2]",WELL THE CHILDREN LOVE IT AFTER AWHILE YOU GET...,4.0,MONKEYS,positive,Apps_for_Android,0.500000,24


In [31]:
df.groupby('aspect')[['helpfullness', 'overall', 'words']].agg(['mean', 'std']).round(2)

helpfullness       overall         words        
                                    mean   std    mean   std    mean     std
aspect                                                                      
Apps_for_Android                    0.22  0.37    3.03  1.43   47.15   50.15
Baby                                0.29  0.42    3.33  1.40  105.34  106.30
Beauty                              0.32  0.41    3.10  1.39   89.99   94.08
Books                               0.43  0.40    3.08  1.35  176.47  201.26
CDs_and_Vinyl                       0.44  0.38    3.11  1.37  171.82  168.08
Cell_Phones_and_Accessories         0.19  0.37    3.06  1.39   93.36  137.57
Clothing_Shoes_and_Jewelry          0.26  0.41    3.11  1.35   66.95   70.27
Digital_Music                       0.53  0.42    4.19  1.11  202.02  190.29
Electronics                         0.43  0.44    3.06  1.41  122.32  137.65
Grocery_and_Gourmet_Food            0.33  0.42    3.53  1.36   98.82   96.96
Health_and_Personal_Care            0.35  0.42    3.09  1.40   95.44  126.29
Home_and_Kitchen                    0.44  0.45    3.08  1.41  104.25  109.82
Kindle_Store                        0.35  0.43    3.07  1.29  111.04  130.97
Movies_and_TV                       0.39  0.38    3.07  1.38  183.70  197.85
Office_Products                     0.29  0.43    4.35  0.93  148.31  163.98
Pet_Supplies                        0.27  0.42    3.35  1.45   90.67   96.37
Sports_and_Outdoors                 0.30  0.42    3.14  1.37   93.90  110.72
Tools_and_Home_Improvement          0.40  0.45    3.90  1.33  110.94  133.73
Toys_and_Games                      0.32  0.43    3.70  1.31  107.52  113.70
Video_Games                         0.41  0.40    3.07  1.37  225.66  267.22

In [32]:
print(df.groupby('aspect')[['helpfullness', 'overall', 'words']].agg(['mean', 'std']).round(2).to_latex(index=True, escape=False))

\begin{tabular}{lrrrrrr}
\toprule
{} & \multicolumn{2}{l}{helpfullness} & \multicolumn{2}{l}{overall} & \multicolumn{2}{l}{words} \\
{} &         mean &   std &    mean &   std &    mean &     std \\
aspect                      &              &       &         &       &         &         \\
\midrule
Apps_for_Android            &         0.22 &  0.37 &    3.03 &  1.43 &   47.15 &   50.15 \\
Baby                        &         0.29 &  0.42 &    3.33 &  1.40 &  105.34 &  106.30 \\
Beauty                      &         0.32 &  0.41 &    3.10 &  1.39 &   89.99 &   94.08 \\
Books                       &         0.43 &  0.40 &    3.08 &  1.35 &  176.47 &  201.26 \\
CDs_and_Vinyl               &         0.44 &  0.38 &    3.11 &  1.37 &  171.82 &  168.08 \\
Cell_Phones_and_Accessories &         0.19 &  0.37 &    3.06 &  1.39 &   93.36 &  137.57 \\
Clothing_Shoes_and_Jewelry  &         0.26 &  0.41 &    3.11 &  1.35 &   66.95 &   70.27 \\
Digital_Music               &         0.53 &  0.42 &   

In [35]:
df.groupby('aspectSentiment')['helpfullness'].count()

aspectSentiment
Apps_for_Android-negative)               20000
Apps_for_Android-neutral)                20000
Apps_for_Android-positive)               20000
Baby-negative)                           17012
Baby-neutral)                            17255
Baby-positive)                           25733
Beauty-negative)                         20000
Beauty-neutral)                          20000
Beauty-positive)                         20000
Books-negative)                          20000
Books-neutral)                           20000
Books-positive)                          20000
CDs_and_Vinyl-negative)                  20000
CDs_and_Vinyl-neutral)                   20000
CDs_and_Vinyl-positive)                  20000
Cell_Phones_and_Accessories-negative)    20000
Cell_Phones_and_Accessories-neutral)     20000
Cell_Phones_and_Accessories-positive)    20000
Clothing_Shoes_and_Jewelry-negative)     20000
Clothing_Shoes_and_Jewelry-neutral)      20000
Clothing_Shoes_and_Jewelry-positive)     200

In [36]:
df['helpfullness'].agg('mean')

0.3493063729114904

In [37]:
df['overall'].agg('mean')

3.314710649331494

In [38]:
df['words'].agg(['mean', 'std'])

mean    122.136397
std     150.979648
Name: words, dtype: float64

In [39]:
t = df['words'].sum()
t

145740233

## Remove rare tokens


The cell below removes/expands contractions

In [2]:
import re
def en_contraction_removal(text: str) -> str:
    apostrophe_handled = re.sub("’", "'", text)
    # from https://gist.githubusercontent.com/tthustla/74e99a00541264e93c3bee8b2b49e6d8/raw/599100471e8127d6efad446717dc951a10b69777/yatwapart1_01.py
    contraction_mapping = {
                    "i.e.": 'for example',
                    "e.g.": 'for example',
                    "youre": "you are",
                    "youll": "you will",
                    "theyre": "they are", "theyll": "they will",
                    "weve": "we have",
                    "shouldnt": "should not",
                    "dont": "do not",
                    "doesnt": "does not", "doesn": "does not",
                    "didnt": "did not",
                    "wasn": "was not",
                    "arent": "are not", "aren": "are not",
                    "aint": "is not", "isnt": "is not", "isn": "is not",
                    "wouldnt": "would not", "wouldn": "would not",
                    "ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                       "here's": "here is",
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have" }
    expanded = ' '.join([contraction_mapping[t.lower()] if t.lower() in contraction_mapping else t for t in apostrophe_handled.split(" ")])
    return expanded

### Word frequency definition
Run the cell below to define the word frequency dict for the dataset. RUn the cell with `clean_text_without_sp` first

In [5]:
path = os.path.join(os.getcwd(), 'data', 'data', 'amazon', 'raw')
df = pd.read_pickle(os.path.join(path, 'dataset_processed.pkl'))


word_frequencies = defaultdict(int)
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    sent = clean_text_without_sp(row).split(" ")
    for t in sent:
        if t == '' or t == ' ':
            continue
        word_frequencies[t.lower()] += 1
word_frequencies


A Jupyter Widget

defaultdict(int,
            {'oh': 20952,
             'how': 176807,
             'my': 1029761,
             'little': 216445,
             'grandson': 3707,
             'loves': 35254,
             'this': 2075925,
             'app': 40085,
             'he': 342547,
             'is': 3183695,
             'always': 82484,
             'asking': 4469,
             'for': 1575876,
             'monkey': 2040,
             'grandma': 1461,
             'has': 398339,
             'tired': 9760,
             'of': 2758116,
             'it': 3366700,
             'long': 119132,
             'before': 117348,
             'finding': 10443,
             'the': 7575353,
             'items': 22762,
             'on': 1140580,
             'each': 76823,
             'page': 20022,
             'that': 1667822,
             'can': 402062,
             'touch': 21136,
             'and': 3958519,
             'activate': 1435,
             'endlessly': 803,
             'entertaining':

### Data Frame creation

In [6]:
d_w_f = {
    'words': list(word_frequencies.keys()),
    'frequencies': list(word_frequencies.values())
}
df_wf = pd.DataFrame.from_dict(d_w_f)
df_wf.sort_values(by=['frequencies'])

,frequencies,words
386879,1,flaxceed
338369,1,^_^b4
338368,1,ultra-relaxed
338367,1,d-out
338366,1,maghic
578622,1,=====old
578623,1,review=======pros
338363,1,2gather
578624,1,considerx-acto
338361,1,chevybtw


### Display some statistics

In [23]:
total_words = df_wf['frequencies'].sum()
tr_1_count = df_wf[df_wf['frequencies'] == 1]
tr_2_count = df_wf[df_wf['frequencies'] <= 2]
tr_3_count = df_wf[df_wf['frequencies'] <= 3]

vocab_size = df_wf['frequencies'].count()
c1 = tr_1_count["words"].count()
c2 = tr_2_count["words"].count()
c3 = tr_3_count["words"].count()

print(f'Vocab Size: {vocab_size}')
print(f'TR-1: {c1}/{total_words} -> Total ratio in sentences: {(c1/total_words)*100}% - Vocab Size: {(c1/vocab_size)*100}%')
print(f'TR-2: {c2}/{total_words} -> Total ratio in sentences: {(c2/total_words)*100}% - Vocab Size: {(c2/vocab_size)*100}%')
print(f'TR-3: {c3}/{total_words} -> Total ratio in sentences: {(c3/total_words)*100}% - Vocab Size: {(c3/vocab_size)*100}%')

Vocab Size: 773759
TR-1: 487243/148129490 -> Total ratio in sentences: 0.3289304513233658% - Vocab Size: 62.970899207634424%
TR-2: 575782/148129490 -> Total ratio in sentences: 0.38870180407696% - Vocab Size: 74.41360940551257%
TR-3: 613798/148129490 -> Total ratio in sentences: 0.4143658362693344% - Vocab Size: 79.32676711999473%


In [24]:
total_words

148129490

### Get statistics for the SP experiment

In [22]:
path = os.path.join(os.getcwd(), 'data', 'data', 'amazon', 'splits')
frames = []

frames.append(pd.read_pickle(os.path.join(path, 'train_sp.pkl')))
frames.append(pd.read_pickle(os.path.join(path, 'train_sp.pkl')))
frames.append(pd.read_pickle(os.path.join(path, 'train_sp.pkl')))
combined_sp_df = pd.concat(frames)




148129490

In [45]:
word_frequencies = defaultdict(int)
for index, row in tqdm(combined_sp_df.iterrows(), total=combined_sp_df.shape[0]):
    sent = clean_text_without_sp(row).split(" ")
    for t in sent:
        if t == '' or t == ' ':
            continue
        word_frequencies[t.lower()] += 1
d_w_f_sp = {
    'words': list(word_frequencies.keys()),
    'frequencies': list(word_frequencies.values())
}
df_wf_sp = pd.DataFrame.from_dict(d_w_f_sp)
df_wf_sp.sort_values(by=['frequencies'])

A Jupyter Widget

,frequencies,words
155371,3,traselvana
205588,3,anti-soccer
205589,3,hughes-
205590,3,oooowwwwww
205591,3,ch-ow-ow-ow-ow
205592,3,oooooooowwwwwwww
205594,3,06-
205595,3,trio-
205596,3,gallant-
205597,3,safer-than-milksop


In [46]:
total_words = df_wf['frequencies'].sum()
tr_1_count = df_wf_sp[df_wf_sp['frequencies'] == 1]
tr_2_count = df_wf_sp[df_wf_sp['frequencies'] <= 2]
tr_3_count = df_wf_sp[df_wf_sp['frequencies'] <= 3]

vocab_size = df_wf_sp['frequencies'].count()
c1 = tr_1_count["words"].count()
c2 = tr_2_count["words"].count()
c3 = tr_3_count["words"].count()

print(f'Vocab Size: {vocab_size}')
print(f'TR-1: {c1}/{total_words} -> Total ratio in sentences: {(c1/total_words)*100}% - Vocab Size: {(c1/vocab_size)*100}%')
print(f'TR-2: {c2}/{total_words} -> Total ratio in sentences: {(c2/total_words)*100}% - Vocab Size: {(c2/vocab_size)*100}%')
print(f'TR-3: {c3}/{total_words} -> Total ratio in sentences: {(c3/total_words)*100}% - Vocab Size: {(c3/vocab_size)*100}%')

Vocab Size: 310743
TR-1: 0/148129490 -> Total ratio in sentences: 0.0% - Vocab Size: 0.0%
TR-2: 0/148129490 -> Total ratio in sentences: 0.0% - Vocab Size: 0.0%
TR-3: 176259/148129490 -> Total ratio in sentences: 0.11898981087425603% - Vocab Size: 56.72179260675221%


In [55]:
new_word_count = df_wf_sp['frequencies'].sum()
#print(f'SP reduction: {new_word_count/total_words}')
print(f'New {new_word_count}\nOld {total_words}')

New 318427872
Old 148129490


In [48]:
1-vocab_size/1594742

0.8051452836885213

### Remove TR-1, TR-2, TR-3
tr-1, 2, 3 only contain tokons which occur at least once

In [7]:
tr_1 = df_wf[df_wf['frequencies'] > 1]
tr_2 = df_wf[df_wf['frequencies'] > 2]
tr_3 = df_wf[df_wf['frequencies'] > 3]
tr_3

,frequencies,words
0,20952,oh
1,176807,how
2,1029761,my
3,216445,little
4,3707,grandson
5,35254,loves
6,2075925,this
7,40085,app
8,342547,he
9,3183695,is


In [18]:
tr_1_s = set(tr_1['words'])
tr_2_s = set(tr_2['words'])
tr_3_s = set(tr_3['words'])



def remove_tokens(row, word_list):
    result = []
    tokens = clean_text_without_sp(row).split(" ")

    for t in tokens:
        if t == ' ' or t == '':
            continue
        if t.lower() in word_list:
            result.append(t)
        else:
            #print(f'Remove {t} to unk')
            result.append('<UNK>')
    return ' '.join(result)

test = {'reviewText': 'This is a text with an unknown word Felix'}
remove_tokens(test, tr_1_s)

'This is a text with an unknown word Felix'

### Actually create new datasets

In [19]:
path = os.path.join(os.getcwd(), 'data', 'data', 'amazon', 'splits')

to_patch = ['test.pkl', 'train.pkl', 'val.pkl', 'test_sp.pkl', 'train_sp.pkl', 'val_sp.pkl']
patches = [tr_1_s, tr_2_s, tr_3_s]

for i, patch in enumerate(patches):
    
    # for the spellcheker only use the first patch
    if i > 0:
        iterator = to_patch[:3]
    else:
        iterator = to_patch
        
    for patch_path in iterator:       
        new_filename = f'{patch_path.replace(".pkl", "")}_TR-{i+1}.pkl'
        print(f'Patching TR-{i+1} on {patch_path} -> {new_filename}')
        patch_df = pd.read_pickle(os.path.join(path, patch_path))
        patch_df['reviewText'] = patch_df.progress_apply(lambda r: remove_tokens(r, patch), axis=1)
        patch_df.to_pickle(os.path.join(path, new_filename))


Patching TR-1 on test.pkl -> test_TR-1.pkl


A Jupyter Widget

Patching TR-1 on train.pkl -> train_TR-1.pkl


A Jupyter Widget

Patching TR-1 on val.pkl -> val_TR-1.pkl


...

A Jupyter Widget

Patching TR-3 on train.pkl -> train_TR-3.pkl


A Jupyter Widget

Patching TR-3 on val.pkl -> val_TR-3.pkl


A Jupyter Widget

### Remove tokens from the complete dataset
(Be careful.. ;))

In [78]:
df['reviewText'] = df.progress_apply(lambda r: remove_tokens(r, tr_1_s), axis=1)
df

Remove Toledo-made to unk
Remove bateria to unk
Remove Felucia to unk
Remove Kashyk to unk
Remove Tattoine to unk
Remove awhileCons-It to unk
Remove Gigaset to unk
Remove C610A to unk
Remove necesaary to unk
Remove answersdefinitely to unk
Remove makesone to unk
Remove dark-brown to unk
Remove incorporatedin to unk
Remove explanationon to unk
Remove unreconized to unk
Remove relistining to unk
Remove Pappa to unk
Remove Californacation to unk
Remove sh#t to unk
Remove sport-mans to unk
Remove leatherand to unk
Remove leatherheavy to unk
Remove Hatti to unk
Remove homebased to unk
Remove novice~ to unk
Remove extrodanariy to unk
Remove bounus to unk
Remove layerd to unk
Remove limbless to unk
Remove lion-hearted to unk
Remove technequies to unk
Remove relams to unk
Remove Skops to unk
Remove Boston-based to unk
Remove buzzed-about to unk
Remove singer-song to unk
Remove Angelakos to unk
Remove Sleepyhead to unk
Remove Bonnaroo to unk
Remove down-pour to unk
Remove medium-medium to unk
R

Remove JaneVI to unk
Remove IIVII to unk
Remove HappensBonus to unk
Remove FeaturesI to unk
Remove PanII to unk
Remove FeaturetteIII to unk
Remove DisneyIV to unk
Remove NeverlandV to unk
Remove GalleryVI to unk
Remove Song-Along to unk
Remove SongVII to unk
Remove StorybookVIII to unk
Remove GameIX to unk
Remove ROMVHS to unk
Remove EXTRASPreviewsI to unk
Remove MulanII to unk
Remove MermaidIII to unk
Remove FlubberIV to unk
Remove MarsV to unk
Remove TimeBonus to unk
Remove ~350-400F to unk
Remove outgassed to unk
Remove orriginally to unk
Remove protion to unk
Remove thiough to unk
Remove sparringly to unk
Remove onb to unk
Remove freezer-- to unk
Remove remounted to unk
Remove antiglare to unk
Remove COOKINGI to unk
Remove TASTETo to unk
Remove pistachios to unk
Remove brined to unk
Remove [Looks to unk
Remove ]INGREDIENTSThe to unk
Remove carb-counter to unk
Remove 49g to unk
Remove SUMMARYGreat to unk
Remove individualizes to unk
Remove Varney to unk
Remove unenergetic to unk
Rem

Remove prevaricate to unk
Remove }}}The to unk
Remove fraudulence to unk
Remove cognitional to unk
Remove Jacobin to unk
Remove adumbrated to unk
Remove Regenburg to unk
Remove [Ernest to unk
Remove II] to unk
Remove Saxe-Gotha to unk
Remove Timoleon to unk
Remove Saxe-Coburg to unk
Remove Saxe-Coburg-Gotha to unk
Remove 1688 to unk
Remove Antients to unk
Remove Antient to unk
Remove Freemasons to unk
Remove Abiff to unk
Remove Jacobinism to unk
Remove prevarications to unk
Remove fromJoe to unk
Remove Harringtonthat to unk
Remove asPiero to unk
Remove Scaruffi to unk
Remove onMusic to unk
Remove masterpieceHejirain to unk
Remove hisPanasonic to unk
Remove ES-LA63-S to unk
Remove 3-position to unk
Remove pschydelia to unk
Remove UGGHS to unk
Remove prison-tight to unk
Remove big--and to unk
Remove tradiionally to unk
Remove peasantry to unk
Remove Golfoid to unk
Remove jumpsuits to unk
Remove Voulez to unk
Remove 1976-80 to unk
Remove diplayed to unk
Remove Eurotrashy to unk
Remove com

Remove BULKAll to unk
Remove accessCONSNot to unk
Remove easeCase to unk
Remove gunshow to unk
Remove Brittax to unk
Remove syrupie to unk
Remove Campbell-Hausfeld to unk
Remove food-gathering to unk
Remove logician to unk
Remove Kunstler to unk
Remove Demon-Haunted to unk
Remove Taner to unk
Remove Edis to unk
Remove goth-punk to unk
Remove cuttingIn to unk
Remove diffirent to unk
Remove #2It to unk
Remove #4My to unk
Remove #4I to unk
Remove #28My to unk
Remove outplayed to unk
Remove HEET to unk
Remove PSL to unk
Remove fitg to unk
Remove Barnacle to unk
Remove wifi-tethering to unk
Remove Boost-crippled to unk
Remove rubbish- to unk
Remove lessons- to unk
Remove Indie-type to unk
Remove retro-influenced to unk
Remove indie-retro to unk
Remove file-format to unk
Remove Spinrite to unk
Remove nothing-is-well-made-anyway to unk
Remove calld to unk
Remove oOOOoohhh to unk
Remove scarrry to unk
Remove 6-week to unk
Remove 24-shot to unk
Remove technoid to unk
Remove spaceThe to unk
Remo

Remove no-fail to unk
Remove colorful2 to unk
Remove freethe to unk
Remove ]Compared to unk
Remove raindeers to unk
Remove re-examining to unk
Remove over-turned to unk
Remove stance-not to unk
Remove non-arbitrary to unk
Remove shelfe to unk
Remove coloered to unk
Remove BAMN to unk
Remove Good-Graphics to unk
Remove loooooooooong to unk
Remove particals to unk
Remove -Climbing to unk
Remove Bad-Character to unk
Remove Ugly-Why to unk
Remove variety-There to unk
Remove growths to unk
Remove brasso to unk
Remove seame to unk
Remove star-power to unk
Remove re-zeroed to unk
Remove 10-15F to unk
Remove ventilators to unk
Remove trached to unk
Remove endotrachial to unk
Remove trachs to unk
Remove DNFs to unk
Remove Viets to unk
Remove adornment to unk
Remove 190X to unk
Remove ballardeer to unk
Remove 26mths to unk
Remove Tippman to unk
Remove Taxol to unk
Remove {second} to unk
Remove sku# to unk
Remove Tolo to unk
Remove Klasky to unk
Remove poiter to unk
Remove iight to unk
Remove doo

...

Remove throbbed to unk
Remove spic-and-span to unk
Remove moistureisez to unk
Remove laves to unk
Remove ]ALSO to unk
Remove languages--those to unk
Remove terminologies to unk
Remove back-lighted to unk
Remove in-country to unk
Remove 2025 to unk
Remove GAMEPLAYTwo to unk
Remove 2-3hr to unk
Remove takeCASTER to unk
Remove 3mins to unk
Remove two-handers to unk
Remove systemRANGED to unk
Remove REPLAYIf to unk
Remove funny-bad to unk
Remove campaign- to unk
Remove chapters- to unk
Remove lock-pick to unk
Remove systemCONS- to unk
Remove unpolished- to unk
Remove parts- to unk
Remove spots- to unk
Remove mega-viruses to unk
Remove revocation to unk
Remove [revocation] to unk
Remove revoking to unk
Remove [techniques] to unk
Remove Revokation to unk
Remove websurfing to unk
Remove 4a-4b to unk
Remove asArkham to unk
Remove Horrorwhich to unk
Remove Mi-Go to unk
Remove higher-than-normal to unk
Remove Larve to unk
Remove each= to unk
Remove happilly to unk
Remove watermellon to unk
Remov

Remove 268-192 to unk
Remove HarperOne to unk
Remove 261 to unk
Remove pre-condition to unk
Remove Testament] to unk
Remove `interview to unk
Remove `Appendix to unk
Remove `supernatural to unk
Remove inerrantly to unk
Remove `inspired to unk
Remove Spicq to unk
Remove sustainer to unk
Remove revealer to unk
Remove `guaranteed to unk
Remove `Those to unk
Remove Ebionite to unk
Remove Marcionite to unk
Remove proto-orthodox to unk
Remove manifestly to unk
Remove non-canonical to unk
Remove `sola to unk
Remove scriptura to unk
Remove `Textus to unk
Remove Receptus to unk
Remove thatmight to unk
Remove milennium to unk
Remove supressor to unk
Remove cocoa-coffee to unk
Remove just-right to unk
Remove pseudo-sugar to unk
Remove grab-n-go to unk
Remove claylike to unk
Remove _supposed_ to unk
Remove befog to unk
Remove veieos to unk
Remove jizz to unk
Remove cummy to unk
Remove Nuun to unk
Remove visualisation to unk
Remove non-gamer to unk
Remove Evil- to unk
Remove type-o to unk
Remove Lu

Remove whole-food to unk
Remove choes to unk
Remove Jesus-- to unk
Remove historian-- to unk
Remove anti-Semite to unk
Remove people-leaders to unk
Remove CNBC to unk
Remove demonify to unk
Remove hallucinated to unk
Remove satan-character to unk
Remove version--one to unk
Remove you-the to unk
Remove JAMA to unk
Remove it--most to unk
Remove JesusFilm to unk
Remove com- to unk
Remove healthfoold to unk
Remove accidenatl to unk
Remove huskier to unk
Remove face-drops to unk
Remove Squier to unk
Remove just-baked to unk
Remove Oxybenzene to unk
Remove theFUNC to unk
Remove microscopically to unk
Remove subscript to unk
Remove EXTREEEEME to unk
Remove laser-pointer to unk
Remove device--one to unk
Remove incredicably to unk
Remove thisstuff to unk
Remove moizterizer to unk
Remove blegian to unk
Remove fantastic- to unk
Remove Latin-inflected to unk
Remove -paid to unk
Remove LOLAs to unk
Remove wardley to unk
Remove waferBetter to unk
Remove wafersCons to unk
Remove fishys to unk
Remove 

Remove Breslin to unk
Remove hats-off to unk
Remove --movieweb-- to unk
Remove longwindedness to unk
Remove ridicluous to unk
Remove enloop to unk
Remove blinki to unk
Remove paddled to unk
Remove verbena-- to unk
Remove 10x20 to unk
Remove machinest to unk
Remove Sesions to unk
Remove moneydon to unk
Remove sugar--this to unk
Remove typicalIRA to unk
Remove Inja to unk
Remove maker--just to unk
Remove `Heroes to unk
Remove Odditiy to unk
Remove future-fascist to unk
Remove future-- to unk
Remove soc-society to unk
Remove Ch-ch-ch-changes to unk
Remove insane-elf to unk
Remove bleaker to unk
Remove Psychodelicate to unk
Remove NOTE>>>THIS to unk
Remove mountain-fresh to unk
Remove Bankston to unk
Remove contracption to unk
Remove atopic to unk
Remove drug-oriented to unk
Remove semi-divine to unk
Remove Easterm to unk
Remove drug-crazed to unk
Remove pell-mell to unk
Remove `Firstlings to unk
Remove `gods to unk
Remove Chthon to unk
Remove Delany to unk
Remove Panshin to unk
Remove det

,helpful,reviewText,overall,summary,sentiment,aspect,aspectSentiment
20965,"[3, 8]",This pot is like the ugly step child in my kit...,3.0,The Ugliest Calphalon Money Can Buy,neutral,Home_and_Kitchen,Home_and_Kitchen-neutral)
4785,"[0, 0]",the snaps open easy and my speed 44 mag loader...,5.0,easy to use,positive,Sports_and_Outdoors,Sports_and_Outdoors-positive)
114490,"[0, 0]",the brush itself is super soft which i love bu...,3.0,bristles do fall out,neutral,Baby,Baby-neutral)
23793,"[0, 0]",So I used to be a Razor man 2 3 blade models ...,5.0,Great for sensitive skin,positive,Health_and_Personal_Care,Health_and_Personal_Care-positive)
45021,"[1, 1]",It is well written and imaginative but a very ...,3.0,Enjoyable read,neutral,Kindle_Store,Kindle_Store-neutral)
15739,"[28, 30]",Again i am given the difficult task of review...,4.0,If you enjoy action platformers and star wars ...,positive,Video_Games,Video_Games-positive)
17571,"[0, 0]",I use these on my Radio Popper wireless flash ...,5.0,"Great price for these, good to stock up.",positive,Tools_and_Home_Improvement,Tools_and_Home_Improvement-positive)
21842,"[0, 0]",This is a great case and low price case the o...,5.0,Good Product Low Price,positive,Cell_Phones_and_Accessories,Cell_Phones_and_Accessories-positive)
137327,"[0, 1]",do not waste your money with these go with pi...,1.0,JUNK!,negative,Electronics,Electronics-negative)
1384,"[0, 0]",Amazing candy I love these things and boy do ...,5.0,Sweet Smarties!!,positive,Grocery_and_Gourmet_Food,Grocery_and_Gourmet_Food-positive)


# Generate 80 - 20 - 10 Splits

In [2]:
path = os.path.join(os.getcwd(), 'data', 'data', 'amazon', 'raw')
df = pd.read_pickle(os.path.join(path, 'dataset_processed.pkl'))
df

,helpful,reviewText,overall,summary,sentiment,aspect
1,"[0, 0]","Oh, how my little grandson loves this app. He'...",5.0,2-year-old loves it,positive,Apps_for_Android
2,"[0, 0]",I found this at a perfect time since my daught...,5.0,Fun game,positive,Apps_for_Android
3,"[3, 4]",My 1 year old goes back to this game over and ...,5.0,We love our Monkeys!,positive,Apps_for_Android
4,"[1, 1]",There are three different versions of the song...,5.0,This is my granddaughters favorite app on my K...,positive,Apps_for_Android
5,"[3, 3]",THis is just so cute and a great app for littl...,5.0,so cute,positive,Apps_for_Android
6,"[3, 4]",I watch my great grandson 4 days a week and it...,5.0,Terrific!,positive,Apps_for_Android
7,"[0, 0]",This app is wild and crazy. Little ones love ...,5.0,Five Little Monkeys,positive,Apps_for_Android
8,"[3, 4]",love love love this app. I was going through d...,5.0,love but to quite,positive,Apps_for_Android
9,"[0, 0]","Very cute, with alot of items to move about. ...",5.0,Cute,positive,Apps_for_Android
10,"[1, 2]",WELL THE CHILDREN LOVE IT AFTER AWHILE YOU GET...,4.0,MONKEYS,positive,Apps_for_Android


In [34]:
# we need an extra attribute to stratify on which is a combination of aspect-sentiment
df['aspectSentiment'] = df.apply(lambda r: f'{r["aspect"]}-{r["sentiment"]})', axis=1)
df

,helpful,reviewText,overall,summary,sentiment,aspect,helpfullness,words,aspectSentiment
1,"[0, 0]","Oh, how my little grandson loves this app. He'...",5.0,2-year-old loves it,positive,Apps_for_Android,0.000000,47,Apps_for_Android-positive)
2,"[0, 0]",I found this at a perfect time since my daught...,5.0,Fun game,positive,Apps_for_Android,0.000000,53,Apps_for_Android-positive)
3,"[3, 4]",My 1 year old goes back to this game over and ...,5.0,We love our Monkeys!,positive,Apps_for_Android,0.750000,43,Apps_for_Android-positive)
4,"[1, 1]",There are three different versions of the song...,5.0,This is my granddaughters favorite app on my K...,positive,Apps_for_Android,1.000000,134,Apps_for_Android-positive)
5,"[3, 3]",THis is just so cute and a great app for littl...,5.0,so cute,positive,Apps_for_Android,1.000000,31,Apps_for_Android-positive)
6,"[3, 4]",I watch my great grandson 4 days a week and it...,5.0,Terrific!,positive,Apps_for_Android,0.750000,60,Apps_for_Android-positive)
7,"[0, 0]",This app is wild and crazy. Little ones love ...,5.0,Five Little Monkeys,positive,Apps_for_Android,0.000000,47,Apps_for_Android-positive)
8,"[3, 4]",love love love this app. I was going through d...,5.0,love but to quite,positive,Apps_for_Android,0.750000,39,Apps_for_Android-positive)
9,"[0, 0]","Very cute, with alot of items to move about. ...",5.0,Cute,positive,Apps_for_Android,0.000000,26,Apps_for_Android-positive)
10,"[1, 2]",WELL THE CHILDREN LOVE IT AFTER AWHILE YOU GET...,4.0,MONKEYS,positive,Apps_for_Android,0.500000,24,Apps_for_Android-positive)


In [14]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.1, random_state=42, stratify=df[['aspectSentiment']])
train, val = train_test_split(train, test_size=0.2, random_state=42, stratify=train[['aspectSentiment']])



In [15]:
val.count()['overall']/df.count()['overall']

0.18000046930336944

In [17]:
train.groupby('sentiment').count()

,helpful,reviewText,overall,summary,aspect,aspectSentiment
sentiment,,,,,,
negative,242913,242913,242913,242913,242913,242913
neutral,251767,251767,251767,251767,251767,251767
positive,364465,364465,364465,364465,364465,364465


In [19]:
path = os.path.join(os.getcwd(), 'data', 'data', 'amazon')

split_path = os.path.join(path, 'splits')
train.to_pickle(os.path.join(split_path, 'train.pkl'))
test.to_pickle(os.path.join(split_path, 'test.pkl'))
val.to_pickle(os.path.join(split_path, 'val.pkl'))

train.to_csv(os.path.join(split_path, 'train.csv'))
test.to_csv(os.path.join(split_path, 'test.csv'))
val.to_csv(os.path.join(split_path, 'val.csv'))

# Preprocessing & Spell Checking

In [3]:
import hunspell


In [76]:
hobj = hunspell.HunSpell('/Library/Spelling/en_US.dic', '/Library/Spelling/en_US.aff')


known_words = ['wirelessly', 'hitman', 'Wal-Mart', 'noob', 'subwoofer', 'WTF', 'Waitrose', '<URL>', 'axe', 'TLDR', 'Coca~Cola', 'NPC', 'sci-fi', 'PS3', 'PSX', 'Clooney', 'Schumacher', 'PS2', 'XBOX']


for w in known_words:
    hobj.add(w)

In [4]:
url_regex = r'(?:http(s)?:\/\/)?[\w.-]+(?:\.[\w\.-]+)+[\w\-\._~:/?#[\]@!\$&\(\)\*\+,;=.]+'

def replace_urls_regex(sentence: str, url_token: str = '<URL>') -> str:
    return re.sub(url_regex, url_token, sentence)

def replace_urls(words, url_token: str = '<URL>'):
    return [url_token if (w.lower().startswith('www') or w.lower().startswith('http')) else w for w in words]

def clean_text_without_sp(row) -> str:
    sent = ''
    try:
        sent = row['reviewText']
        #print(sent)
        to_remove = [',', '(', ')', ':', '?', '&', '/', '*', '!', ';', '"', '.', '+']
        for tr in to_remove:
            sent = sent.replace(tr, ' ')

        sent = sent.replace('€™', "'")
        sent = sent.replace('�', "'")
        sent = en_contraction_removal(sent)
        sent = sent.replace("'", ' ')
        sent = replace_urls_regex(sent)    
    except Exception as err:
        print('Could not clean sentence: ' + str(err))
        
        try:
            sent = row['reviewText']
            return sent
        except Exception as err:
            print('Could not get sentence from row. Returning empty sentence: ' + str(err))
            return ''
    return sent

def clean_text(row) -> str:
    sent = ''
    try:
        sent = spellcheck_sentence(row)
    except Exception as err:
        print('Could not spellcheck sentence: ' + str(err))
        
        try:
            sent = row['reviewText']
            return sent
        except Exception as err:
            print('Could not get sentence from row. Returning empty sentence: ' + str(err))
            return ''
    return sent
        

def spellcheck_sentence(row) -> str:
    sent = row['reviewText']
    #print(sent)
    to_remove = [',', '(', ')', ':', '?', '&', '/', '*', '!']
    for tr in to_remove:
        sent = sent.replace(tr, ' ')
        
    sent = sent.replace('€™', "'")
    sent = sent.replace('�', "'")
    sent = en_contraction_removal(sent)
    sent = sent.replace("'", ' ')
    sent = replace_urls_regex(sent)


    tokens = sent.split(' ')
    result = []
    for t in tokens:
        if t == ' ':
            continue
            
        correct = True
        try:
            correct = hobj.spell(t)
        except Exception as err:
            print('Could not get spell checking for token ' + str(t))
            # do not keep token in case of error
            continue
            
        if not correct:
            suggestions = hobj.suggest(t)
            if not suggestions:
                result.append(t)
            else:
                if suggestions[0] == 'e':
                    result.append(t)
                    continue
                result.append(suggestions[0])
                #print(f'{t} -> {suggestions[0]}')
        else:
            result.append(t)
    return ' '.join(result)
        
#spellcheck_sentence('This is a tset with a wong wod. Adn now anotheer one why does this notjn workd')
test = {'reviewText': "This is a test which I'm doing. I can't stress that I don't care..."}
clean_text_without_sp(test)

'This is a test which i am doing  I cannot stress that I do not care   '

In [12]:
splits = ['train', 'val', 'test']
path = os.path.join(os.getcwd(), 'data', 'data', 'amazon')

for s in splits:
    print('Split: ' + str(s))
    fn = os.path.join(path, s + '.pkl')
    df = pd.read_pickle(fn)
    df['reviewText'] = df.progress_apply(clean_text, axis=1)
    
    fn = os.path.join(path, s + '_sp.pkl')
    df.to_pickle(fn)
    df.to_csv(os.path.join(path, s + '_sp.csv'), sep='|', index=False)

Split: train


Could not get spell checking for token ch eat
Could not get spell checking for token a i
Could not get spell checking for token c nsecutive
Could not get spell checking for token wh istles
Split: val


Split: test


In [8]:
df.to_pickle(fn)
df.to_csv(os.path.join(path, s + '_sp.csv'), sep='|', index=False)

### Apply token removals on all splits and the spell checked one

make sure to run cells necessary for the cells below before